# Ziad Al-Ziadi

## Testing our MLP and SVM models with the EmotionRecognition function

In the preceding section, we define the EmotionRecognition function to use with the SVM and MLP models we built and trained previously. The models will be loaded in as pickle files.

In [ ]:
!pip install opencv-python==4.4.0.46

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import random
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
####### PLEASE DRAG THE "CV_Submission" FROM YOUR "Shared with me" INTO YOUR "My Drive" OTHERWISE THIS WON'T WORK #######

import os

MY_OWN_PATH = "/content/drive/MyDrive/CV_Submission/"
MY_OWN_PATH = "CV_Submission"
FULL_PATH = os.path.join("drive", "My Drive", MY_OWN_PATH)
print(os.listdir(FULL_PATH))

In [ ]:
zip_path = "/content/drive/MyDrive/CV_Submission/CW_Dataset.zip"
!cp "{zip_path}" .
!yes|unzip -q CW_Dataset.zip

In [ ]:
# Creating our dataframe 

test_image_path = "/content/test"
test_txt = "/content/labels/list_label_test.txt"

colnames = ["Image_id", "Label"]
test_lab = pd.read_csv(test_txt, names = colnames, header = None, delim_whitespace = True)

test_lab["Image_id"] = test_lab["Image_id"].apply(lambda i: i[:-4])
test_lab["Image_id"] = test_lab["Image_id"] + "_aligned.jpg"
test_lab["Label"] = test_lab["Label"].apply(lambda i: i - 1)
test_lab["Label"] = test_lab["Label"].apply(lambda i: i + 1)

In [ ]:
# Defining our function which takes a model and test dataframe

def EmotionRecognition(model , test_path):

########################## SVM-HOG AND MLP-HOG ##########################

  if model == SVM_HOG or model == MLP_HOG:
    test_img = []
    labels = []

    for i, row in test_path.iterrows():
      img_path = "/content/test/"
      img_name = row[0]
      lbl = row[1]
      img_path += str(img_name)
      img = cv2.imread(img_path)
      hog_des = hog(img, orientations = 8, pixels_per_cell = (14, 14),
                  cells_per_block = (1,1), visualize = False)

      if hog_des is not None:
        test_img.append(hog_des)
        labels.append(lbl)
  
    X_test = np.array(test_img)

    predicted = model.predict(X_test).tolist()

    fig, axes = plt.subplots(1, 4, figsize=(14, 7), sharex=True, sharey=True)
    ax = axes.ravel()
    emo = {1: "Surprise", 2: "Fear", 3: "Disgust", 4: "Happiness", 5: "Sadness", 6: "Anger", 7: "Neutral"}

    # Reformatting our image data for easier plotting
    test_img = []

    for i, row in test_path.iterrows():
      img_path = "/content/test/"
      img_name = row[0]
      lbl = row[1]
      img_path += str(img_name)
      img = cv2.imread(img_path)
      if img is not None:
        test_img.append(img)

    random.shuffle(test_img)
    for i in range(4):
      ax[i].imshow(test_img[i])
      ax[i].set_title(f'Image Label: {emo[labels[i]]} \n Model Prediction: {emo[predicted[i]]}')
      ax[i].set_axis_off()
    fig.tight_layout()
    plt.show()

########################## SVM-SIFT AND MLP-SIFT ##########################

  elif model == MLP_SIFT or model == SVM_SIFT:

    test_img = []
    labels = []

    for i, row in test_path.iterrows():
      img_path = "/content/test/"
      img_name = row[0]
      lbl = row[1]
      img_path += str(img_name)
      img = cv2.imread(img_path)
      sift = cv2.SIFT_create()
      kp, des = sift.detectAndCompute(img, None)
      if des is not None:
        test_img.append(des)
        labels.append(lbl)

    sift_hist = np.vstack(test_img)
    cluster = 70
    batch_size = 178194
    k_means = MiniBatchKMeans(n_clusters = cluster, batch_size=batch_size).fit(sift_hist)

    img_list = []
    for x in test_img:
      hist = np.zeros(cluster)
      idx = k_means.predict(x)
      
      for y in idx:
        hist[y] = hist[y] + (1 / len(x))
      img_list.append(hist)
  
    X_test = np.array(img_list)

    predicted = model.predict(X_test).tolist()

    fig, axes = plt.subplots(1, 4, figsize=(14, 7), sharex=True, sharey=True)
    ax = axes.ravel()
    emo = {1: "Surprise", 2: "Fear", 3: "Disgust", 4: "Happiness", 5: "Sadness", 6: "Anger", 7: "Neutral"}

    # Reformatting our image data for easier plotting
    test_img = []

    for i, row in test_path.iterrows():
      img_path = "/content/test/"
      img_name = row[0]
      lbl = row[1]
      img_path += str(img_name)
      img = cv2.imread(img_path)
      if img is not None:
        test_img.append(img)

    random.shuffle(test_img)
    for i in range(4):
      ax[i].imshow(test_img[i])
      ax[i].set_title(f'Image Label: {emo[labels[i]]} \n Model Prediction: {emo[predicted[i]]}')
      ax[i].set_axis_off()
    fig.tight_layout()
    plt.show()

########################## SVM-ORB AND MLP-ORB ##########################

  elif model == MLP_ORB or model == SVM_ORB:
    
    test_img = []
    labels = []

    for i, row in test_path.iterrows():
      img_path = "/content/test/"
      img_name = row[0]
      lbl = row[1]
      img_path += str(img_name)
      img = cv2.imread(img_path)
      orb = cv2.ORB_create()
      kp, des = orb.detectAndCompute(img, None)
      if des is not None:
        test_img.append(des)
        labels.append(lbl)

    orb_hist = np.vstack(test_img)
    cluster = 70
    batch_size = 178194
    k_means = MiniBatchKMeans(n_clusters = cluster, batch_size=batch_size).fit(orb_hist)

    img_list = []

    for x in test_img:
      hist = np.zeros(cluster)
      idx = k_means.predict(x)
      
      for y in idx:
        hist[y] = hist[y] + (1 / len(x))
      img_list.append(hist)
  
    X_test = np.array(img_list)

    predicted = model.predict(X_test).tolist()

    fig, axes = plt.subplots(1, 4, figsize=(14, 7), sharex=True, sharey=True)
    ax = axes.ravel()
    emo = {1: "Surprise", 2: "Fear", 3: "Disgust", 4: "Happiness", 5: "Sadness", 6: "Anger", 7: "Neutral"}

    # Reformatting our image data for easier plotting
    test_img = []

    for i, row in test_path.iterrows():
      img_path = "/content/test/"
      img_name = row[0]
      lbl = row[1]
      img_path += str(img_name)
      img = cv2.imread(img_path)
      if img is not None:
        test_img.append(img)

    random.shuffle(test_img)
    for i in range(4):
      ax[i].imshow(test_img[i])
      ax[i].set_title(f'Image Label: {emo[labels[i]]} \n Model Prediction: {emo[predicted[i]]}')
      ax[i].set_axis_off()
    fig.tight_layout()
    plt.show()

  else:
    print("Model not found, check your spelling")

In [ ]:
########################### Loading our MLP Models ##########################

# MLP with SIFT
with open("/content/drive/MyDrive/CV_Submission/Test Functions/sift_mlp.pkl", "rb") as mlp_sift:
    mlpsift = pickle.load(mlp_sift)

# MLP with ORB
with open("/content/drive/MyDrive/CV_Submission/Test Functions/orb_mlp.pkl", "rb") as mlp_orb:
    mlporb = pickle.load(mlp_orb)

# MLP with HOG
with open("/content/drive/MyDrive/CV_Submission/Test Functions/hog_mlp.pkl", "rb") as mlp_hog:
    mlphog = pickle.load(mlp_hog)

########################### Loading our SVM Models ##########################

#SVM with SIFT
with open("/content/drive/MyDrive/CV_Submission/Test Functions/sift_svm.pkl", "rb") as svm_sift:
    svmsift = pickle.load(svm_sift)

#SVM with ORB
with open("/content/drive/MyDrive/CV_Submission/Test Functions/orb_svm.pkl", "rb") as svm_orb:
    svmorb = pickle.load(svm_orb)

#SVM with HOG
with open("/content/drive/MyDrive/CV_Submission/Test Functions/hog_svm.pkl", "rb") as svm_hog:
    svmhog = pickle.load(svm_hog)

SVM_SIFT = svmsift
SVM_ORB = svmorb
SVM_HOG = svmhog

MLP_SIFT = mlpsift
MLP_ORB = mlporb
MLP_HOG = mlphog

testPath = test_lab

In [ ]:
# Testing our EmotionalRecognition function - input a model variable from above to test

EmotionRecognition(SVM_HOG ,testPath)